In [3]:
import numpy as np
import pandas as pd
import scipy
from scipy.spatial import distance
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from scipy.special import rel_entr

In [211]:
Predicted = pd.read_csv("./SpatialDWLS/Outputs/output_Sim23.csv")
Target=pd.read_csv("./Data/Sim23/target.csv")

In [212]:
Predicted

,Unnamed: 0,cell_ID,NK,CD4_T,Alveolar_Type_2,CD8_T,Monocytes,Macrophage,Alveolar_Type_1,Endothelial,...,Fibroblast,Mast,B_cells,Muscle_cells,Proliferating_NK_T,Ciliated,Plasma,Club cell,Basal,Proliferating_Macrophage
0,1,0,0.000000,0.0,0.000000,0.00000,0.000000,0.936913,0.000000,0.063087,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000e+00
1,2,1,0.000000,0.0,0.352380,0.00000,0.000000,0.490898,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,1.567216e-01
2,3,2,0.053075,0.0,0.057705,0.00000,0.000000,0.889220,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,4.780407e-20
3,4,3,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.741227,...,0.000000,0.000000,0.258773,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000e+00
4,5,4,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,0.225289,0.126626,...,0.000000,0.110118,0.000000,0.0,0.0,0.0,0.0,0.116973,0.420995,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,995,0.000000,0.0,0.247725,0.00000,0.000000,0.676280,0.000000,0.000000,...,0.075996,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000e+00
996,997,996,0.181392,0.0,0.000000,0.00000,0.069625,0.748983,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000e+00
997,998,997,0.000000,0.0,0.000000,0.00000,0.000000,0.213666,0.000000,0.114320,...,0.000000,0.153003,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,5.190111e-01
998,999,998,0.000000,0.0,0.217523,0.00000,0.000000,0.346347,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.436130,0.000000,0.000000e+00


In [213]:
Target

,Unnamed: 0,Endothelial,DC,B_cells,Macrophage,CD8_T,CD4_T,Muscle_cells,Club cell,Alveolar_Type_2,Basal,Proliferating_Macrophage,Proliferating_NK_T,NK,Monocytes,Mast,Ciliated,Fibroblast,Alveolar_Type_1,Plasma
0,0,3.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
1,1,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0
3,3,8.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,4,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
996,996,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
997,997,4.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
998,998,0.0,0.0,1.0,2.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [214]:
Predicted.drop(columns="Unnamed: 0", inplace=True)
Predicted.drop(columns="cell_ID", inplace=True)


In [215]:
Target.drop(columns="Unnamed: 0", inplace=True)

In [216]:
for i in range(len(Target)):
    Target.loc[i,:]=Target.loc[i,:]/Target.loc[i,:].sum()

In [217]:
for i in range(len(Predicted)):
    Predicted.loc[i,:]=Predicted.loc[i,:]/Predicted.loc[i,:].sum()
 

In [218]:
columns_p=[]
for i in range(Predicted.shape[1]):
    columns_p.append(Predicted.columns[i])
    

In [219]:
columns_t=[]
for i in range(Target.shape[1]):
    columns_t.append(Target.columns[i])
    

In [220]:
columns_p=sorted(columns_p)
columns_t=sorted(columns_t)

In [221]:
Predicted=Predicted[columns_p]
Target=Target[columns_t]

In [222]:
Target=Target.to_numpy()
Predicted=Predicted.to_numpy()

## Pearson Correlation

In [223]:
num_celltypes=Predicted.shape[1]
d = np.zeros(num_celltypes)
for i in range(0,num_celltypes):
    p=Predicted[:,i]
    t=Target[:,i]
    prs=(np.corrcoef(p,t,rowvar=False))
    d[i]=prs[0,1]

In [224]:
np.mean(np.nan_to_num(d))

0.7826571346053013

In [225]:
d

array([0.87340487, 0.91780997, 0.70719675, 0.76623808, 0.72215667,
       0.69507383, 0.93642441, 0.6575607 , 0.72690085, 0.8411196 ,
       0.87014244, 0.85029018, 0.8852738 , 0.72327176, 0.84239434,
       0.80384464, 0.93131632, 0.51075813, 0.60930823])

## Jenson Shannon Divergence

In [226]:
jns=distance.jensenshannon(Predicted,Target,axis=1)
jns=jns[~np.isinf(jns).squeeze()]
np.mean(jns)

0.31240391314654664

## F1 Score, Precision, Recall

In [227]:
thresholds={0.001,0.01,0.1,0.2,0.3}

In [228]:
d = pd.DataFrame(0, index=np.arange(len(thresholds)), columns=["Precision","Recall","F1-score"])
d.index=thresholds
for t in thresholds:
    Predicted_logical=(Predicted>t).astype(int)
    Target_logical=(Target>t).astype(int)
    d.loc[t,"Precision"]=precision_score(Target_logical,Predicted_logical,average="micro")
    d.loc[t,"Recall"]=recall_score(Target_logical,Predicted_logical,average="micro")
    d.loc[t,"F1-score"]=f1_score(Target_logical,Predicted_logical,average="micro")

In [229]:
d

,Precision,Recall,F1-score
0.100,0.849491,0.737394,0.789483
0.200,0.791246,0.628903,0.700795
0.300,0.639553,0.562018,0.598284
0.010,0.801915,0.814590,0.808203
0.001,0.790047,0.815502,0.802573
